# Wrapping a template library

A template library is a library where there are only template classes that can be instantiated.
Wrapping such libraries therefore requires **AutoWIG** to be able to consider various *C++* template classes instantiations during the `Parse` step.
The **Standard Template Library (STL)** library is a *C++* library that provides a set of common *C++* template classes such as containers and associative arrays.
These classes can be used with any built-in or user-defined type that supports some elementary operations (e.g. copying, assignment).
It is divided in four components called algorithms, containers, functional and iterators.
**STL** containers (e.g. `std::vector`, `std::set`) are used in many *C++* libraries.
In such a case, it does not seem relevant that every wrapped *C++* library contains wrappers for usual **STL** containers (e.g. `std::vector< double >`, `std::set< int >`).
We therefore proposed *Python* bindings for sequence containers (i.e. `pair`, `array`, `vector`, `deque`, `forward_list` and `list` of the `std` namespace) and associative containers (`set`, `multiset`, `map`, `multimap`, `unordered_set`, `unordered_multiset`, `unordered_map` and `unordered_multimap` of the `std` namespace).
These template instantiations are done for *C++* fundamental types (`bool`, `signed char`, `unsigned char`, `char`, `wchar_t`, `int` (with sign modifiers `signed` and `signed` combined or not with size modifiers `short`, `long` and `long long`), `float`, `double`, `long double`) and strings (`string`, `wstring` of the `std` namespace).
For ordered associative containers both `std::less` and `std::greater` comparators are used.

First, clone the **PySTL**  repository into the **PySTL** directory.

In [3]:
%%bash
git clone https://github.com/StatisKit/PySTL.git

fatal: destination path 'PySTL' already exists and is not an empty directory.


As presented below, in order to wrap a template library, the user needs to write headers containing aliases for desired template class instantiations.

In [2]:
%%bash
pygmentize PySTL/src/cpp/vector.h

#include <vector>
#include <string>

namespace pystl
{
    namespace vector
    {
        typedef std::vector< bool > Bool;
        typedef std::vector< char > Char;
        typedef std::vector< signed char > SignedChar;
        typedef std::vector< unsigned char > UnsignedChar;
        typedef std::vector< int > Int;
        typedef std::vector< short > Short;
        typedef std::vector< short int > ShortInt;
        typedef std::vector< long > Long;
        typedef std::vector< long int > LongInt;
        typedef std::vector< long long > LongLong;
        typedef std::vector< long long int > LongLongInt;
        typedef std::vector< signed > Signed;
        typedef std::vector< signed  int > SignedInt;
        typedef std::vector< signed short > SignedShort;
        typedef std::vector< signed short int > SignedShortInt;
        typedef std::vector< signed long > SignedLong;
        typedef std::vector< signed long int > SignedLongInt;
        typedef std::vector< signed long long >

We import **AutoWIG** and create an empty Abstract Semantic Graph (ASG).

In [3]:
import autowig
asg = autowig.AbstractSemanticGraph()

ImportError: No module named pkgtk.plugin

We need then to install the *C++* headers. This is done using the `cpp` target in **SCons**.

In [12]:
%%bash
scons cpp -C PySTL

scons: Entering directory `/home/pfernique/Desktop/AutoWIG/doc/examples/PySTL'
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
scons: building terminated because of errors.


scons: *** [/usr/include/pystl] /usr/include/pystl: Permission denied


Once the headers habe been installed in the system, we parse headers with relevant compilation flags.

In [ ]:
from path import path
autowig.parser.plugin = 'pyclanglite'
asg = autowig.parser(asg,
                     (path(sys.prefix)/'include'/'pystl').files('*.h'),
                     flags = ['-x', 'c++', '-std=c++11',
                              '-D__STDC_LIMIT_MACROS', '-D__STDC_CONSTANT_MACROS',
                              '-I' + str((prefix/'include').abspath())],
                     libpath = prefix/'lib'/'libclang.so',
                     bootstrap = False,
                     silent = True)

Since most of **AutoWIG** guidelines are respected, the `default` `controller` implementation is thus suitable.

In [ ]:
autowig.controller.plugin = 'default'
asg = autowig.controller(asg)

In [ ]:
autowig.generator.plugin = 'boost_python_pattern'
for pattern in ['vector']:
    wrappers = autowig.generator(asg,
                  pattern = '(class |struct |union |enum |)::std::vector< .* >',
                  module = './PySTL/src/py/' + pattern + '/_' + pattern + '.cpp',
                  decorator = './PySTL/src/py/stl/' + pattern + '/_' + pattern + '.py',
                  closure = True)
    wrappers.write()